In [1]:
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
import torch
from torch.nn import CrossEntropyLoss
from torch.utils.data import Dataset
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback, get_scheduler
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np
from accelerate import Accelerator

#### Import Dataset

In [2]:
import pandas as pd

root_path = "."
df = pd.read_csv(f'{root_path}/data/rating_auto_label_sentiment_two_classes.csv')
df = df[df['sentiment'].isin(['POSITIVE', 'NEGATIVE'])]

#### Step 1: Split the Data into Train, Test, and Eval Sets
We can use sklearn to split the data.

In [3]:
splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

for train_idx, temp_idx in splitter.split(df, df['sentiment']):
    train_df = df.iloc[train_idx]
    temp_df = df.iloc[temp_idx]

# Split temp_df into test and eval (50% each)
test_df, eval_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df['sentiment'], random_state=42)

print(f"Train size: {len(train_df)}, Test size: {len(test_df)}, Eval size: {len(eval_df)}")

Train size: 8369, Test size: 1046, Eval size: 1047


### init tokenizer

In [4]:
# Step 3: Initialize Tokenizer
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

c:\Users\ceffendy\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ceffendy\.cache\huggingface\hub\models--distilbert-base-uncased-finetuned-sst-2-english. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

c:\Users\ceffendy\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


#### Data Preparation (Tokenization)
We need to tokenize the text data and convert it into PyTorch tensors.

In [5]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from torch.utils.data import Dataset
import torch
import numpy as np
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

# Define the SentimentDataset
class SentimentDataset(Dataset):
    def __init__(self, df, tokenizer, max_length=128):
        self.df = df
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        text = row['review_text']
        label = 1 if row['sentiment'] == 'POSITIVE' else 0

        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        
        input_ids = encoding['input_ids'].squeeze()
        attention_mask = encoding['attention_mask'].squeeze()

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Create datasets
train_dataset = SentimentDataset(train_df, tokenizer)
test_dataset = SentimentDataset(test_df, tokenizer)
eval_dataset = SentimentDataset(eval_df, tokenizer)

# Handle Class Imbalance with Weighted Loss
class_counts = df['sentiment'].value_counts().to_dict()
total_samples = len(df)
weights = {
    0: total_samples / class_counts['NEGATIVE'],
    1: total_samples / class_counts['POSITIVE']
}
class_weights = torch.tensor([weights[0], weights[1]], dtype=torch.float)

# Initialize Model
model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased-finetuned-sst-2-english",
    num_labels=2
)
model.config.problem_type = "single_label_classification"

# Define Training Arguments
training_args = TrainingArguments(
    output_dir="./model_checkpoints",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=3,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    load_best_model_at_end=True,
    no_cuda=True,  # Force training on CPU
    disable_tqdm=False,  # Enable progress bar
    report_to="none",  # Disable wandb logging
)

# Define Metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# Custom Trainer to handle class weights
class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.get("labels")
        # Forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # Compute custom loss with class weights
        loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights.to(model.device))
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss

# Initialize Trainer with Early Stopping Callback
trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

# Train the Model
trainer.train()

# Evaluate the Best Model
eval_results = trainer.evaluate(eval_dataset)
print("Evaluation Results:", eval_results)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

c:\Users\ceffendy\anaconda3\Lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
c:\Users\ceffendy\anaconda3\Lib\site-packages\transformers\training_args.py:1540: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(


  0%|          | 0/1572 [00:00<?, ?it/s]

{'loss': 0.2204, 'grad_norm': 8.525217056274414, 'learning_rate': 1.3638676844783715e-05, 'epoch': 0.95}


  0%|          | 0/66 [00:00<?, ?it/s]

...

{'eval_loss': 0.34871822595596313, 'eval_accuracy': 0.9340974212034384, 'eval_f1': 0.9478458049886621, 'eval_precision': 0.93026706231454, 'eval_recall': 0.9661016949152542, 'eval_runtime': 98.0416, 'eval_samples_per_second': 10.679, 'eval_steps_per_second': 0.673, 'epoch': 2.0}
{'loss': 0.0516, 'grad_norm': 0.1546747237443924, 'learning_rate': 9.160305343511451e-07, 'epoch': 2.86}


  0%|          | 0/66 [00:00<?, ?it/s]

{'eval_loss': 0.3410106301307678, 'eval_accuracy': 0.9379178605539638, 'eval_f1': 0.9503437738731857, 'eval_precision': 0.9424242424242424, 'eval_recall': 0.9583975346687211, 'eval_runtime': 73.4666, 'eval_samples_per_second': 14.251, 'eval_steps_per_second': 0.898, 'epoch': 3.0}
{'train_runtime': 6181.0627, 'train_samples_per_second': 4.062, 'train_steps_per_second': 0.254, 'train_loss': 0.12268722739838461, 'epoch': 3.0}


  0%|          | 0/66 [00:00<?, ?it/s]

Evaluation Results: {'eval_loss': 0.19172503054141998, 'eval_accuracy': 0.9312320916905444, 'eval_f1': 0.9451219512195121, 'eval_precision': 0.9351432880844646, 'eval_recall': 0.9553158705701078, 'eval_runtime': 72.348, 'eval_samples_per_second': 14.472, 'eval_steps_per_second': 0.912, 'epoch': 3.0}


#### Save the Trained Model
Save the final model to disk.

In [15]:
import zipfile
import os
import shutil
import IPython.display as display
model_name = "fine_tuned_model"
output_folder = f"./output/{model_name}"
model.save_pretrained(output_folder)
tokenizer.save_pretrained(output_folder)



# Create a zip file from the final_model folder and save it to the Lakehouse path
with zipfile.ZipFile(f"./output/{model_name}.zip", "w", zipfile.ZIP_DEFLATED) as zipf:
    for root, dirs, files in os.walk(output_folder):
        for file in files:
            file_path = os.path.join(root, file)
            zipf.write(file_path, os.path.relpath(file_path, output_folder))


print("Model has been zipped and saved to output foder successfully.")


Model has been zipped and saved to output foder successfully.


In [ ]:
import zipfile
import os
import shutil
import IPython.display as display
model_name = "fine_tuned_model"
output_folder = f"./output/{model_name}"
model.save_pretrained(output_folder)
tokenizer.save_pretrained(output_folder)



# Create a zip file from the final_model folder and save it to the Lakehouse path
with zipfile.ZipFile(f"./output/{model_name}.zip", "w", zipfile.ZIP_DEFLATED) as zipf:
    for root, dirs, files in os.walk(output_folder):
        for file in files:
            file_path = os.path.join(root, file)
            zipf.write(file_path, os.path.relpath(file_path, output_folder))


print("Model has been zipped and saved to output foder successfully.")


### Inference

In [22]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizerFast
import torch
import torch.nn.functional as F  # To use softmax

# Path to the saved model and tokenizer
model_path = './output/fine_tuned_model'

# Load the model and tokenizer from the saved path
model = DistilBertForSequenceClassification.from_pretrained(model_path)
tokenizer = DistilBertTokenizerFast.from_pretrained(model_path)

# Set the model to evaluation mode (disable dropout, etc.)
model.eval()

def preprocess_text(text, tokenizer, max_length=128):
    encoding = tokenizer(
        text,
        truncation=True,
        padding='max_length',
        max_length=max_length,
        return_tensors='pt'
    )
    return encoding

def predict_sentiment(text, model, tokenizer):
    # Preprocess the input text
    encoding = preprocess_text(text, tokenizer)
    
    # Perform inference
    with torch.no_grad():
        outputs = model(**encoding)
        logits = outputs.logits  # Get the raw prediction scores
        
        # Apply softmax to get probabilities
        probabilities = F.softmax(logits, dim=-1)  # Apply softmax along the last dimension
        predicted_class = torch.argmax(probabilities, dim=-1)  # Get the predicted class (0 for negative, 1 for positive)
    
    # Map the predicted class to sentiment label
    sentiment = 'POSITIVE' if predicted_class.item() == 1 else 'NEGATIVE'
    
    # Return sentiment and probabilities for each class
    positive_prob = probabilities[0][1].item()  # Probability for POSITIVE class
    negative_prob = probabilities[0][0].item()  # Probability for NEGATIVE class
    
    return sentiment, positive_prob, negative_prob




In [24]:
# Sample sentences (positive and negative examples)
sample_sentences = [
    "I absolutely love this movie! It was amazing.",
    "This movie was terrible, I hated every second of it.", 
    "while this movie is not intended for everyone, it is good for someone has no brain", 
    "let's watch it only when it is free to watch, i will not pay for it",
    'A worthy contender for the Animated film of 2024', 
    'No plot at all. But if you are looking for a good laugh. You will not find that either.'
]


# Perform inference on each sample sentence
for sentence in sample_sentences:
    sentiment, positive_prob, negative_prob = predict_sentiment(sentence, model, tokenizer)
    print(f"Sentence: {sentence}\nPredicted Sentiment: {sentiment}")
    print(f"POSITIVE: {positive_prob:.4f}, NEGATIVE: {negative_prob:.4f}\n")


Sentence: I absolutely love this movie! It was amazing.
Predicted Sentiment: POSITIVE
POSITIVE: 0.9998, NEGATIVE: 0.0002

Sentence: This movie was terrible, I hated every second of it.
Predicted Sentiment: NEGATIVE
POSITIVE: 0.0004, NEGATIVE: 0.9996

Sentence: while this movie is not intended for everyone, it is good for someone has no brain
Predicted Sentiment: NEGATIVE
POSITIVE: 0.0683, NEGATIVE: 0.9317

Sentence: let's watch it only when it is free to watch, i will not pay for it
Predicted Sentiment: NEGATIVE
POSITIVE: 0.0671, NEGATIVE: 0.9329

Sentence: A worthy contender for the Animated film of 2024
Predicted Sentiment: POSITIVE
POSITIVE: 0.9996, NEGATIVE: 0.0004

Sentence: No plot at all. But if you are looking for a good laugh. You will not find that either.
Predicted Sentiment: NEGATIVE
POSITIVE: 0.0017, NEGATIVE: 0.9983

